###1) job_csv-parquet

In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext, SparkConf
from awsglue.context import GlueContext
from awsglue.job import Job

## @params: [JOB_NAME]
args = getResolvedOptions(sys.argv, ['JOB_NAME'])

sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

# Lendo o arquivo CSV da camada RAW do bucket S3
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("sep", "|") \
    .load("s3://projeto-compass-filmes/Raw/Local/CSV/Movies/2022/05/02/movies.csv")

# Filtrando as linhas com base na lista de IDs de filmes do IMDB
lista_franquias = ['tt0120737', 'tt0167261', 'tt0167260', 'tt0903624', 'tt1170358', 'tt2310332', 'tt2310332', 'tt0133093', 'tt0234215', 'tt0242653', 'tt10838180', 'tt0088247', 'tt0103064', 'tt0181852', 'tt0438488', 'tt1340138', 'tt6450804', 'tt0088763', 'tt0096874', 'tt0099088', 'tt0076759', 'tt0080684', 'tt0086190', 'tt0120915', 'tt0121765', 'tt0121766', 'tt2488496', 'tt2527336', 'tt2527338', 'tt0079501', 'tt0082694', 'tt0089530', 'tt1392190', 'tt0078748', 'tt0090605', 'tt0103644', 'tt0118583', 'tt1446714', 'tt2316204']
movies = df.filter(df['id'].isin(lista_franquias))

# Removendo colunas
colunas_para_remover = ['tituloOriginal', 'anoLancamento', 'tempoMinutos',
                        'genero', 'notaMedia', 'numeroVotos',
                        'generoArtista','titulosMaisConhecidos']
movies_franquias = movies.drop(*colunas_para_remover)

# Salvando o arquivo como movies.parquet no bucket S3
movies_franquias.write.mode('overwrite').parquet("s3://projeto-compass-filmes/TRT/Movies/movies.parquet")

job.commit()

### job_json-parquet

In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql.functions import lit

## @params: [JOB_NAME]
args = getResolvedOptions(sys.argv, ['JOB_NAME'])

sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

# Insira seu código aqui

# Lendo o arquivo JSON da camada RAW do bucket S3
df = spark.read.json("s3://projeto-compass-filmes/Raw/TMDB/JSON/2023/06/26/franquias.json")

# Adicionando a coluna "dt" com o valor da data desejada
df = df.withColumn("dt", lit("2023-06-26"))

# Escrevendo o arquivo em formato Parquet na camada TRUSTED do bucket S3, particionado por data
df.write.partitionBy("dt").parquet("s3://projeto-compass-filmes/TRT/franquias.parquet")

job.commit()